<a href="https://colab.research.google.com/github/jagatabhay/S4/blob/master/S5_Assignment_File2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# https://pytorch.org/docs/stable/torchvision/transforms.html
train_transforms = transforms.Compose([ transforms.ToTensor(), 
                                        transforms.Normalize((0.1307,) , (0.3081,))])

test_transforms = transforms.Compose([ transforms.ToTensor(), 
                                      transforms.Normalize((0.1307,),(0.3081,))])

train = datasets.MNIST('./data',
                        train = True , 
                        download = True , 
                        transform = train_transforms )

test = datasets.MNIST('./data', 
                       train = False , 
                       download = True , 
                       transform = test_transforms )


seed = 1
torch.manual_seed(seed)
print(torch.manual_seed(seed))

device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )
print("Device : ", device)

if torch.cuda.is_available():
  torch.cuda.manual_seed(seed)

Device :  cuda


In [0]:
batch_size = 100

train_loader = torch.utils.data.DataLoader(train , 
                                           batch_size = batch_size , 
                                           shuffle = True , 
                                           pin_memory = True , 
                                           num_workers = 1 )

test_loader = torch.utils.data.DataLoader(test ,
                                          batch_size = batch_size ,
                                          shuffle = True ,
                                          pin_memory = True ,
                                          num_workers = 1)

In [0]:
train_data = train.train_data
#print("Train Data 1 : ", train_data)

train_data = train.transform( train_data.numpy() )
#print("Train Data 2 : ",train_data)

/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [0]:
print(" -Numpy Shape   : " , train.train_data.cpu().numpy().shape )
print(" -Tensor Shape  : " , train.train_data.size() )
print(" -Minimum       : " , torch.min(train_data) )
print(" -Maximun       : " , torch.max(train_data) )
print(" -Mean          : " , torch.mean(train_data) )
print(" -Varian        : " , torch.var(train_data) )
print(" -STD Deviation : " , torch.std(train_data) )

 [ Data Stats and View ] 
 -Numpy Shape   :  (60000, 28, 28)
 -Tensor Shape  :  torch.Size([60000, 28, 28])
 -Minimum       :  tensor(-0.4242)
 -Maximun       :  tensor(2.8215)
 -Mean          :  tensor(0.0009)


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:55: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


 -Varian        :  tensor(1.0001)
 -STD Deviation :  tensor(1.0000)


In [0]:
dataiter = iter(train_loader)
images , labels = dataiter.next()

print("Image Shape  : " , images.shape )
print("Labels Shape : " , labels.shape )

Image Shape  :  torch.Size([100, 1, 28, 28])
Labels Shape :  torch.Size([100])


In [0]:
class Net(nn.Module):
    def _init_(self):
        super(Net, self)._init_()
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        #self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.conv5 = nn.Conv2d(256, 512, 3)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.conv7 = nn.Conv2d(1024, 10, 3)
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=10,kernel_size=(3,3),padding=0), #26
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=10),
                                   #nn.Dropout(0.1)
                                   )

        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=10,out_channels=16,kernel_size=(3,3),padding=0), #24
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=16),
                                   #nn.Dropout(0.1)
        )
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=16,out_channels=20,kernel_size=(3,3),padding=0), #22
                                   nn.BatchNorm2d(num_features=20),
                                   #nn.Dropout(0.1)
                                   )                          
        
        self.pool1 =  nn.MaxPool2d(2,2)
         
        
        
        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=20,out_channels=10,kernel_size=(1,1),padding=0), #11
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=10),
                                   nn.Dropout(0.1)
                                   ) 
        
        #self.pool2 =  nn.MaxPool2d(2,2)

        self.conv5 = nn.Sequential(nn.Conv2d(in_channels=10,out_channels=20,kernel_size=(3,3),padding=0), #9
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=20),
                                   #nn.Dropout(0.1)
                                   ) 
        
        self.conv6 = nn.Sequential(nn.Conv2d(in_channels=20,out_channels=16,kernel_size=(3,3),padding=0), #7
                                   nn.ReLU(),
                                   nn.BatchNorm2d(num_features=16),
                                   #nn.Dropout(0.1)
                                   ) 
        self.conv7 = nn.Sequential(nn.Conv2d(in_channels=16,out_channels=10,kernel_size=(1,1),padding=0) #5
                                   #nn.ReLU()
                                   #nn.BatchNorm2d(num_features=10)
                                   #nn.DroupOut2d()
                                   )
        self.gap  =  nn.Sequential(nn.AvgPool2d(kernel_size=7)) 

        
        
        
        self.dropout = nn.Dropout(0.1)

              
        



    def forward(self, x):
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        #x = x.view(-1, 10)
        x = self.conv1(x)
        x = self.conv2(x)
        
        x = self.conv3(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = self.conv4(x)
        #x = self.pool2(x)
        x = self.conv5(x)
        x = self.conv6(x)
        
        x = self.conv7(x)
        x = self.gap(x)
        

        x = x.view(-1, 10)
        



        return F.log_softmax(x)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

AttributeError: ignored

In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)                        # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1,9):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    print("Iteration No : ",epoch)
    print('\n')

  0%|          | 0/600 [00:00<?, ?it/s]


Test set: Average loss: 0.0902, Accuracy: 9768/10000 (97.68%)

Iteration No :  1




  0%|          | 0/600 [00:00<?, ?it/s]


Test set: Average loss: 0.0551, Accuracy: 9837/10000 (98.37%)

Iteration No :  2




  0%|          | 0/600 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0298, Accuracy: 9901/10000 (99.01%)

Iteration No :  6




  0%|          | 0/600 [00:00<?, ?it/s]


Test set: Average loss: 0.0348, Accuracy: 9905/10000 (99.05%)

Iteration No :  7




loss=0.009450649842619896 batch_id=599: 100%|██████████| 600/600 [00:18<00:00, 32.09it/s]



Test set: Average loss: 0.0340, Accuracy: 9899/10000 (98.99%)

Iteration No :  8


